In [10]:
import numpy as np
import sys
import traceback
from IPython.display import clear_output
from itertools import groupby
from time import sleep
import random as rd


class ConnectN:

    MAX_DEPTH = 3
    
    ME, AI = 1, -1

    def __init__(self, connect_n=4, sx=7, sy=6):

        assert (
            sx >= connect_n and sy >= connect_n
        ), f"invalid dimensions / n-connect number: ({sx}, {sy}) => ({connect_n}) ?!"

        self.sx, self.sy, self.connect_n = sx, sy, connect_n

    def get_new_board(self):
        return np.zeros(shape=(self.sy, self.sx), dtype=int)

    @staticmethod
    def print(c4):

        sy, sx = c4.shape
        symbols = {0: " . ", 1: " o ", -1: " x "}

        header = [f"{x:02d}." for x in range(sx)]
        print(*header)
        for i in range(sy):
            L = []
            for j in range(sx):
                v = symbols[c4[i, j]]
                L.append(f"{v:3}")
            print(*L, flush=True)

    @staticmethod
    def is_valid_column(board, col):
        _, sx = board.shape
        return col >= 0 and col < sx and 0 in board[:, col]

    @staticmethod
    def play(board, j, is_max_player):

        sy, sx = board.shape
        if j and not ConnectN.is_valid_column(board, j):
            raise RuntimeError(f"invalid column {j}", file=sys.stderr)

        bc = board.copy()  # don't modify the input

        def drop(j):
            for i in range(sy - 1, -1, -1):
                if bc[i, j] == 0:
                    bc[i, j] = ConnectN.AI if is_max_player else ConnectN.ME
                    return True
            return False

        if j is not None:
            st = drop(j)
            assert st
        else:
            for j in rd.sample(range(sx), sx):
                st = drop(j)
                if st:
                    break
        return bc

    def run(self):

        is_max_player = False  # e.g. is AI playing?
        is_game_over = False
        c4 = self.get_new_board()
        while True:
            try:
                ConnectN.print(c4)
                if is_game_over:
                    break
                    
                clear_output(wait=(not is_max_player))
                if is_max_player:
                    score, j = self.alphabeta(
                        c4, ConnectN.MAX_DEPTH, -np.inf, np.inf, is_max_player)
                    print(f"best column={j} (score:{score})")
                else:
                    j = int(input("play:"))

                c4 = ConnectN.play(c4, j, is_max_player)

                if ConnectN.is_winning_move(c4, self.connect_n, is_max_player):
                    is_game_over = True
                    pyr = "AI" if is_max_player else "PLAYER"
                    print(f"*** Game Over! Winner: {pyr} ***")

                is_max_player = not is_max_player
            except Exception as e:
                print(e, file=sys.stderr)

    @staticmethod
    def line_count_max_connect(line, val):
        max_connect = 0
        for l in [list(v) for _, v in groupby(line)]:
            if np.equal(l, val).all():
                max_connect = max(max_connect, len(l))
        return max_connect

    @staticmethod
    def is_winning_move(board, connect_n, is_max_player=True):

        sy, sx = board.shape
        assert (
            connect_n <= sx and connect_n <= sy
        ), f"invalid dimensions / n-connect number: ({sx}, {sy}) => ({connect_n}) ?!"

        val = ConnectN.AI if is_max_player else ConnectN.ME

        # check for a horizontal win - count consecutive 1s
        for i in range(sy):
            line = board[i, :]
            if not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) == connect_n:
                return True

        # check for a vertical win
        for j in range(sx):
            line = board[:, j]
            if not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) == connect_n:
                return True

        # check for a diagonal win
        for offset in range(-(sy - 1), sx):
            line = np.diagonal(board, offset)
            if len(line) < connect_n or not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) == connect_n:
                return True

        # check for an anti-diagonal win
        for offset in range(-(sy - 1), sx):
            line = np.diagonal(np.flipud(board), offset)  # vertical flip
            if len(line) < connect_n or not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) == connect_n:
                return True

        return False

    @staticmethod
    def is_last_move(node, connect_n):
        nb_turn_left = len(list(filter(lambda x: x == 0, node.flatten())))
        return True if nb_turn_left == 1 else False

    @staticmethod
    def score(node, connect_n, is_max_player):

        sy, sx = node.shape
        score = 0

        pyr, opp = (ConnectN.AI, ConnectN.ME) if is_max_player else (ConnectN.ME, Connect.AI)

        def get_updated_score(line, score):
            """ Compute score based on heuristic function """
            
            def count(val): return len([x for x in line if x == val])
            
            nb_connect_pyr = count(pyr)
            nb_connect_opp = count(opp)
            nb_connect_empty = count(0)
            
            #print(pyr, line, 'pyr:', nb_connect_pyr, 'opp:', nb_connect_opp, 'empty:', nb_connect_empty)
            if nb_connect_pyr >= 2 and not nb_connect_opp:  # only empty left
                score += np.exp(nb_connect_pyr)
            elif nb_connect_opp >= 2 and not nb_connect_pyr:
                score -= np.exp(nb_connect_opp)
                
            return score

        # check for a horizontal win
        for i in range(sy):

            line = node[i, :]

            if not any(line):
                continue
                
            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i+connect_n]
                score = get_updated_score(sub_line, score)

        # check for a vertical win
        for j in range(sx):

            line = node[:, j]

            if not any(line):
                continue

            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i + connect_n]
                score = get_updated_score(sub_line, score)
                
        # check for a diagonal win
        for offset in range(-(sy - 1), sx):
            
            line = np.diagonal(node, offset)
            
            if len(line) < connect_n or not any(line):
                continue
                
            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i + connect_n]
                score = get_updated_score(sub_line, score)

        # check for an anti-diagonal win
        for offset in range(-(sy - 1), sx):
            
            line = np.diagonal(np.flipud(node), offset)  # vertical flip
            
            if len(line) < connect_n or not any(line):
                continue
                
            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i + connect_n]
                score = get_updated_score(sub_line, score)

        return score

    def alphabeta(self, board, depth, alpha, beta, is_max_player=True):

        sy, sx = board.shape

        # leaf node: score it
        if depth == 0 or ConnectN.is_last_move(board, self.connect_n):
            return ConnectN.score(board, self.connect_n, True), None

        if is_max_player:
            value = -np.inf
            for k in range(0, sx):
                if 0 in board[:, k]:
                    b = ConnectN.play(board, k, True)
                    value_new, _ = self.alphabeta(
                        b, depth - 1, alpha, beta, False)

                    if value_new > value:  # maximize value
                        value = value_new
                        best_col = k

                    if value >= beta:  # beta pruning
                        break

                    alpha = max(alpha, value)  # no fail-soft

        else:
            value = np.inf
            for k in range(0, sx):
                if 0 in board[:, k]:
                    b = ConnectN.play(board, k, False)
                    value_new, _ = self.alphabeta(
                        b, depth - 1, alpha, beta, True)

                    if value_new < value:  # minimize value
                        value = value_new
                        best_col = k

                    if alpha >= value:  # alpha pruning
                        break

                    beta = min(beta, value)  # no fail-soft

        return value, best_col


connect_four = ConnectN()
connect_four.run()

best column=0 (score:237.09857770434516)
*** Game Over! Winner: AI ***
00. 01. 02. 03. 04. 05. 06.
 .   .   x   o   .   .   . 
 .   x   o   x   .   x   . 
 x   x   x   x   .   x   x 
 o   o   o   x   .   o   o 
 o   o   x   o   o   x   o 
 o   x   o   o   o   x   x 
